## 1. TensorFlow Hub
- https://tfhub.dev/
- 모델을 조정 및 배포 가능한 학습된 머신러닝 모델의 저장소
- 단 몇 줄의 코드만으로 CNN과 BERT와 같은 학습된 모델을 재사용할 수 있음
- 한글
    - https://www.tensorflow.org/?hl=ko → https://www.tensorflow.org/hub/tutorials/tf2_image_retraining?hl=ko

> MODEL 가져오기\
tensorflow hub → image → efficientnet_v2 → Efficientnetv2 B0_feature_vector → copy URL

> Efficientnetv2 B0_feature_vector  VS   Efficientnetv2 B0_classification\
=> 정답이 없는 분류  VS  정답이 있는 분류

In [ ]:
# !pip install tensorflow_hub

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.applications import EfficientNetB0    # 위의 사이트에 쓰여 있는 모델이름 import

import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow_hub as hub

In [3]:
model = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2'),    # 모델 선택
    tf.keras.layers.Dense(1, activation='sigmoid')    # 1: 결과 개수
])

model.build([None, 256, 256, 3])    # None : 배치 사이즈

adam = keras.optimizers.Adam(lr=0.0001)

model.compile(
    optimizer=adam,
    loss='binary_crossentropy',
    metrics='accuracy'
)

model.summary()

# 8_캐라스 어플리케이션에서 결과와 비교
# 여기서는 version_2를 사용하고 있음 => 성능 조금 더 높게 나올 듯
# Total params 증가

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              5919312   
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 5,920,593
Trainable params: 1,281
Non-trainable params: 5,919,312
_________________________________________________________________


c:\program files\python37\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [4]:
# 7_Albunmentation

import albumentations as A

class Augmentation:
    def __init__(self, size, mode='train'):
        if mode == 'train':
            self.transform = A.Compose([
                # 수평
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(
                    p=0.5,
                    shift_limit=0.05,
                    scale_limit=0.05,
                    rotate_limit=15
                ),
                # 구멍을 최대 8개 Dropout
                A.CoarseDropout(
                    p=0.5,
                    max_holes=8,
                    max_height=int(0.1 * size),
                    max_width=int(0.1 * size)
                ),
                A.RandomBrightnessContrast(p=0.2)
            ])
        
    # **kwargs : 매개변수 개수 상관없이 받아서 리스트 처리
    def __call__(self, **kwargs):
        if self.transform :
            augmented = self.transform(**kwargs)
            img = augmented['image']
            return img
            
        

In [5]:
import math

class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, fold, image_size, mode='train',  shuffle=True):
        self.batch_size = batch_size
        self.fold = fold
        self.image_size = image_size
        self.mode = mode
        self.shuffle = shuffle
        
        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
            
            
        # https://github.com/tensorflow/models/issues/3134
        # 파일 이슈 -> 삭제
        # Egyptian_Mau_14, 139, 145, 156, 167, 177, 186, 191;
        # Abyssinian_5, 34. 121, 116
        
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'Abyssinian_121',
            'Abyssinian_116'
        ]
        
        # isin : 포함을 의미
        self.df = self.df[~self.df['file_name'].isin(invalid_filenames)]    # invalid_filenames을 file_name으로 찾아서 그것을 제외한 데이터프레임 생성
        self.transform = Augmentation(image_size, mode)
        
        self.on_epoch_end()
        
    # sample() : 전체 row에서 몇 %의 데이터를 return 할 것인지 설정
    # frac : 데이터프레임으로 부터 특정 비율로 무작위 표본 추출을 하고 싶으면 0~1 사이의 부동소수점 입력
    # reset_index() : drop=True 옵션을 설정하면 인덱스 열을 보통의 자료열로 설정하는 것이 아니라 버림
    # 여기서 인데스 열이란 데이터프레임 맨 앞 열에 생성되는 인덱스 열을 말함
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
            
    # len() 함수를 재구현  (공유파일 첫 달에서 스페셜매소드 파일에서 다뤘음)
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
    
    # 배열을 쓸 때 자동으로 불러지는 것
    def __getitem__(self, idx):
        start = idx * self.batch_size
        end = (idx + 1) * self.batch_size
        data = self.df.iloc[start:end]
        batch_x, batch_y = self.get_data(data)
        return np.array(batch_x), np.array(batch_y)
        
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        # iterrows
        # 데이터프레임에서 row의 개수만큼 반복
        # 튜플로 리턴[index, 행정보(Series)]
        for _, r in data.iterrows():             
            file_name = r['file_name']
            image = cv2.imread(f'D:/data_ai/5_AI/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (self.image_size, self.image_size))
            
            if self.mode == 'train':
                image = image.astype('uint8')
                image = self.transform(image=image)
                
            image = image.astype('float32')
            image = image / 255    # 정규화
            
            label = int(r['species']) - 1    # 밑에서 class_name 인덱스와 맞추기위해 (-1)해줌
            
            batch_x.append(image)
            batch_y.append(label)
            
        return batch_x, batch_y

In [6]:
csv_path = 'D:/data_ai/5_AI/kfolds.csv'

train_generator = DataGenerator(
    # 옵션
    batch_size = 128, 
    csv_path = csv_path, 
    fold = 1, 
    image_size = 256, 
    mode = 'train', 
    shuffle = True
)

valid_generator = DataGenerator(
    # 옵션
    batch_size = 128, 
    csv_path = csv_path, 
    fold = 1, 
    image_size = 256, 
    mode = 'val', 
    shuffle = True
)

In [7]:
history = model.fit(
    train_generator,
    validation_data = valid_generator,
    epochs=10,
    verbose=1
)

# 실행은 되지만 느림
# 중간에 중지시킴

Epoch 1/10
46/46 [==============================] - 403s 9s/step - loss: 0.5769 - accuracy: 0.7580 - val_loss: 0.5380 - val_accuracy: 0.8372
Epoch 2/10
14/46 [========>.....................] - ETA: 3:32 - loss: 0.5097 - accuracy: 0.8471

KeyboardInterrupt: 